In [88]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [74]:
link1 = "./BD/df_basic_name_clean.csv"
link2 = "./BD/df_t_akas_clean.csv"
link3 = "./BD/df_t_basics_clean.csv"
link4 = "./BD/df_t_crew_clean.csv"
link5 = "./BD/df_t_episode_clean.csv"
link6 = "./BD/df_t_principals_clean.csv"
link7 = "./BD/df_t_ratings_clean.csv"
link8 = "./BD/tmdb_full.csv"
link9 = "./BD/df_tID.csv"
link10 = "./BD/df_nID.csv"

In [20]:
df_basic_name = pd.read_csv(link1)

In [12]:
df_t_akas = pd.read_csv(link2)

In [6]:
df_t_basics = pd.read_csv(link3)

In [75]:
df_t_crew = pd.read_csv(link4)

In [ ]:
df_t_episode = pd.read_csv(link5nope)

In [5]:
df_t_principals = pd.read_csv(link6)

In [16]:
df_t_ratings = pd.read_csv(link7)

In [4]:
df_tmdb = pd.read_csv(link8, low_memory = False)

In [5]:
df_tID = pd.read_csv(link9)

In [63]:
df_nID = pd.read_csv(link10)

On veut 2 df: un df avec index tconst et un avec index nconst
1.  Commençons par df_tID

In [ ]:
# Prépartion table title basics en premier car elle permet de selectionner les 'movies' et les années souhaitées
df_tb_tconst = df_t_basics[(df_t_basics['titleType'] == 'movie') & (df_t_basics['startYear'] >= 1950) & (df_t_basics['isAdult'] == 0)]
df_tb_tconst = df_tb_tconst[['tconst', 'primaryTitle', 'startYear', 'runtimeMinutes', 'genres']]

#Préparation table title principals
df_ptconst = df_t_principals.groupby('tconst').agg(list).reset_index()
df_ptconst = df_ptconst.drop('category', axis = 1)

#Premier merge avec title basic en pivot
df_tID = pd.merge(df_tb_tconst, df_ptconst, how = 'left', on='tconst')

#2e merge avec akas deja nettoyé
df_tID = pd.merge(df_tID, df_t_akas, how = 'left', left_on='tconst', right_on='titleId')

#3e merge avec ratings déjà nettoyé
df_tID = pd.merge(df_tID, df_t_ratings, how = 'left', on='tconst')

#4e merge avec les directors de title crew
df_t_crew_fusion = df_t_crew[['tconst', 'directors']]
df_tID = pd.merge(df_tID, df_t_crew_fusion how = 'left', on='tconst')


#Renommage des colonnes
df_tID = df_tID.rename(columns={'tconst' : 't_ID', 'primaryTitle' : 'Titre_Principal', 'startYear' : 'Année', 'genres' : 'Genres', 'runtimeMinutes' : 'Durée', 'nnconst' : 'n_ID_liste', 'title' : 'Titres_possibles', 'averageRating' : 'Note_moyenne', 'numVotes' : 'Nb_votants', 'directors' : 'Réalisateurs'})

#Drop colonnes doublons
df_tID = df_tID.drop(['tconst', 'titleId'], axis = 1)

#Réorganisation des colonnes
df_tID = df_tID[['t_ID', 'Titre_Principal', 'Année', 'Réalisateurs', 'n_ID_liste', 'Durée', 'Genres', 'Note_moyenne', 'Nb_votants', 'Titres_possibles']]

#Trie par année
df_tID = df_tID.sort_values('Année')

#reset index
df_tID= df_tID.reset_index().drop('index', axis = 1)

#Export
df_tID.to_csv('df_tID.csv', index = False)

In [65]:
df_tID

,t_ID,Titre_Principal,Année,Réalisateurs,n_ID_liste,Durée,Genres,Note_moyenne,Nb_votants,Titres_possibles
0,tt0011801,Tötet nicht mehr,2019.0,['nm0681726'],"['nm0681726', 'nm0692612', 'nm0726256', 'nm077...",NaN,"['Action', 'Crime']",NaN,NaN,"['Tötet nicht mehr', 'Misericordia - Tötet nic..."
1,tt0015414,La tierra de los toros,2000.0,['nm0615736'],"['nm0147437', 'nm0615736', 'nm0615736', 'nm061...",60.0,['\\N'],5.4,17.0,"['La tierra de los toros', 'La terre des taure..."
2,tt0015724,Dama de noche,1993.0,['nm0529960'],"['nm0844752', 'nm0869732', 'nm0194720', 'nm065...",102.0,"['Drama', 'Mystery', 'Romance']",6.3,31.0,"['Dama de noche', 'Dama de noche', 'Éjszakai g..."
3,tt0021617,Arizona Territory,1950.0,['nm0289297'],"['nm0934290', 'nm0167395', 'nm0766912', 'nm060...",56.0,['Western'],6.0,61.0,"['Arizona Territory', 'Arizona Territory', 'Cu..."
4,tt0022064,Lebbra bianca,1951.0,['nm0871077'],"['nm0709790', 'nm0561755', 'nm0623443', 'nm048...",80.0,['Drama'],5.2,62.0,"['Lebbra bianca', 'A Lebre Branca', 'Brief Rap..."
...,...,...,...,...,...,...,...,...,...,...
517165,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,2015.0,"['nm9272490', 'nm9272491']","['nm9272513', 'nm10538557', 'nm10538558', 'nm1...",57.0,['Documentary'],NaN,NaN,['Rodolpho Teóphilo - O Legado de um Pioneiro'...
517166,tt9916680,De la ilusión al desconcierto: cine colombiano...,2007.0,['nm0652213'],"['nm0033355', 'nm0127882', 'nm0133349', 'nm105...",100.0,['Documentary'],NaN,NaN,['De la ilusión al desconcierto: cine colombia...
517167,tt9916706,Dankyavar Danka,2013.0,['nm7764440'],"['nm1778107', 'nm2585097', 'nm5697682', 'nm327...",NaN,['Comedy'],7.7,9.0,"['Dankyavar Danka', 'Dankyavar Danka']"
517168,tt9916730,6 Gunn,2017.0,['nm10538612'],"['nm6096005', 'nm0059461', 'nm13233318', 'nm48...",116.0,['Drama'],7.0,12.0,"['6 Gunn', '६ गुण']"


2. Ensuite un df_nID

In [ ]:
#Tri de la table title basic pour avoir les tconst correspondant aux films
df_tb_nconst = df_tb_tconst = df_t_basics[(df_t_basics['titleType'] == 'movie') & (df_t_basics['startYear'] >= 1950) & (df_t_basics['isAdult'] == 0)]
df_tb_nconst = df_tb_nconst[['tconst']]
df_tb_nconst = df_tb_nconst.reset_index()

# Préparation table title principals avec seulement les acteurs
df_t_principals_fusion = df_t_principals[(df_t_principals['category'] == 'actor') | (df_t_principals['category'] == 'actress')]
df_t_principals_fusion = df_t_principals_fusion.drop('category', axis = 1)

#Premier groupby tconst pour merge avec les tconst triés de title basics
df_t_principals_fusion.groupby('tconst').agg(list).reset_index()

#Premier merge par tconst
df_nID = pd.merge(df_t_principals_fusion, df_tb_nconst, how = 'left', on = 'tconst')

#Explode de la table mergée pour refaire le bon groupby cette fois par nconst
df_nID = df_nID.explode('nconst')
df_nID = df_nID.groupby('nconst').agg(list).reset_index()
df_nID = df_nID.drop('index', axis = 1)

#Ajouter des colonnes d'infos de l'acteur et une colonne nb de films:
df_basic_name_fusion = df_basic_name[['nconst', 'primaryName', 'birthYear', 'deathYear']]
df_nID = pd.merge(df_nID, df_basic_name_fusion, how = 'left', on = 'nconst')
df_nID['Nb_films'] = df_nID['tconst'].apply(lambda x : len(x))

#On renomme les colonnes
df_nID = df_nID.rename(columns={'nconst' : 'n_ID', 'tconst' : 'films_joués', 'primaryName' : 'Nom', 'birthYear' : 'Année_naissance', 'deathYear' : 'Année_décès'})

#Plus qu'a exporter!
df_nID.to_csv('df_nID.csv', index = False)

In [66]:
df_nID

,n_ID,films_joués,Nom,Année_naissance,Année_décès,Nb_films
0,nm0000001,"['tt0023926', 'tt0024025', 'tt0025164', 'tt002...",Fred Astaire,1899.0,1987.0,64
1,nm0000002,"['tt0037382', 'tt0037610', 'tt0038355', 'tt003...",Lauren Bacall,1924.0,2014.0,79
2,nm0000003,"['tt0044881', 'tt0046200', 'tt0047607', 'tt004...",Brigitte Bardot,1934.0,NaN,39
3,nm0000004,"['tt0076816', 'tt0077147', 'tt0077621', 'tt007...",John Belushi,1949.0,1982.0,105
4,nm0000005,"['tt0043048', 'tt0064793', 'tt0064897', 'tt006...",Ingmar Bergman,1918.0,2007.0,11
...,...,...,...,...,...,...
3432237,nm9993700,['tt8744178'],Sexy Angel,NaN,NaN,1
3432238,nm9993701,['tt8736744'],Sanjai Kuriakose,NaN,NaN,1
3432239,nm9993703,"['tt11212278', 'tt26598427']",James Craigmyle,NaN,NaN,2
3432240,nm9993708,"['tt11697102', 'tt11702702', 'tt11772842', 'tt...",Eli Bevins,NaN,NaN,5


3. Enfin nettoyage de la table TMDB qui sera notre base pour le système de recommandation

In [73]:
df_t_crew

NameError: name 'df_t_crew' is not defined

In [ ]:
df_tID['Nom_réalisateur']=df_tID['Réalisateurs'].apply

,t_ID,Titre_Principal,Année,Réalisateurs,n_ID_liste,Durée,Genres,Note_moyenne,Nb_votants,Titres_possibles
0,tt0011801,Tötet nicht mehr,2019.0,['nm0681726'],"['nm0681726', 'nm0692612', 'nm0726256', 'nm077...",NaN,"['Action', 'Crime']",NaN,NaN,"['Tötet nicht mehr', 'Misericordia - Tötet nic..."
1,tt0015414,La tierra de los toros,2000.0,['nm0615736'],"['nm0147437', 'nm0615736', 'nm0615736', 'nm061...",60.0,['\\N'],5.4,17.0,"['La tierra de los toros', 'La terre des taure..."
2,tt0015724,Dama de noche,1993.0,['nm0529960'],"['nm0844752', 'nm0869732', 'nm0194720', 'nm065...",102.0,"['Drama', 'Mystery', 'Romance']",6.3,31.0,"['Dama de noche', 'Dama de noche', 'Éjszakai g..."
3,tt0021617,Arizona Territory,1950.0,['nm0289297'],"['nm0934290', 'nm0167395', 'nm0766912', 'nm060...",56.0,['Western'],6.0,61.0,"['Arizona Territory', 'Arizona Territory', 'Cu..."
4,tt0022064,Lebbra bianca,1951.0,['nm0871077'],"['nm0709790', 'nm0561755', 'nm0623443', 'nm048...",80.0,['Drama'],5.2,62.0,"['Lebbra bianca', 'A Lebre Branca', 'Brief Rap..."
...,...,...,...,...,...,...,...,...,...,...
517165,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,2015.0,"['nm9272490', 'nm9272491']","['nm9272513', 'nm10538557', 'nm10538558', 'nm1...",57.0,['Documentary'],NaN,NaN,['Rodolpho Teóphilo - O Legado de um Pioneiro'...
517166,tt9916680,De la ilusión al desconcierto: cine colombiano...,2007.0,['nm0652213'],"['nm0033355', 'nm0127882', 'nm0133349', 'nm105...",100.0,['Documentary'],NaN,NaN,['De la ilusión al desconcierto: cine colombia...
517167,tt9916706,Dankyavar Danka,2013.0,['nm7764440'],"['nm1778107', 'nm2585097', 'nm5697682', 'nm327...",NaN,['Comedy'],7.7,9.0,"['Dankyavar Danka', 'Dankyavar Danka']"
517168,tt9916730,6 Gunn,2017.0,['nm10538612'],"['nm6096005', 'nm0059461', 'nm13233318', 'nm48...",116.0,['Drama'],7.0,12.0,"['6 Gunn', '६ गुण']"


In [ ]:
#L'énoncé nous demandant d'afficher l'affiche du film, on commence par supprimer les lignes qui ne renseignent pas le lien de l'affiche (45413/309572)
df_tmdb_clean = df_tmdb.dropna(subset=['poster_path'])

#Il reste 1 ligne qui renvoie à un film pour public adulte, restons courtois et supprimons la!
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean['adult'] == False]

#La colonne 'release_date' doit être au format datetime pour filtrer les films > 1950 (suppression de 27335 lignes sur 264158 (10%)) et les classer:
df_tmdb_clean["release_date"] = pd.to_datetime(df_tmdb_clean["release_date"])
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean["release_date"].dt.year > 1950]
df_tmdb_clean = df_tmdb_clean.sort_values('release_date')

#On va maintenant sélectionner et réaranger les colonnes pertinentes:
#Seulement 22316/236823 avec un budget > 0 : colonne non pertinente 
#Seulement 14537/236823 avec un revenue > 0 : colonne non pertinente 
df_tmdb_clean = df_tmdb_clean[['imdb_id', 'title', 'original_title', 'original_language', 'release_date', 'genres', 'spoken_languages',  'runtime', 'production_countries', 'vote_average', 'vote_count', 'popularity', 'overview', 'status', 'poster_path']]

In [ ]:
#A discuter mais cela me semble pertinent avec nos choix
df_tmdb_clean[(df_tmdb_clean['original_language'] == 'fr') | (df_tmdb_clean['original_language'] == 'en')]


,imdb_id,title,original_title,original_language,release_date,genres,spoken_languages,runtime,production_countries,vote_average,vote_count,popularity,overview,status,poster_path
254855,tt1857649,Celery Stalks at Midnight,Celery Stalks at Midnight,en,1951-01-01,"['Animation', 'Music']",['xx'],3,['US'],5.2,4,1.400,Early 'visual music' film by John Whitney.,Released,/5EBflggrs5qC1wkjljPf84QZdBW.jpg
83118,tt0276544,Survival Under Atomic Attack,Survival Under Atomic Attack,en,1951-01-01,['Documentary'],['en'],9,['US'],0.0,0,0.600,A Civil Defense film that focuses on what the ...,Released,/nTkOhT4EYW59BRdLFGXFROZRDlp.jpg
262146,tt2304747,Dead Horizons,Les Horizons morts,fr,1951-01-01,['Drama'],['xx'],8,['FR'],5.4,9,0.600,Les Horizons Morts is Jacques Demy's first sur...,Released,/ugHD4En27OCdlQW6LSCDmFYqJT2.jpg
224307,tt0042590,I'll Get You for This,I'll Get You for This,en,1951-01-01,"['Drama', 'Crime']",['en'],87,['GB'],6.3,3,0.956,"Nick, an American gambler, arrives in San Paol...",Released,/poqm1FhjuGJQ1H2CSrwXqhDMz5s.jpg
137284,tt0043146,The Wonder Kid,The Wonder Kid,en,1951-01-01,"['Drama', 'Adventure']",['en'],85,"['AT', 'GB']",0.0,0,0.600,"Bobby Henrey, the amazing juvenile star of The...",Released,/8RYCY5l5U88CaqNXxKUQ6r1V54j.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110786,tt1830809,Spaceless,Spaceless,en,2025-12-15,['Science Fiction'],['en'],0,['US'],0.0,0,2.463,A man wakes up inside a spacesuit tumbling hel...,Planned,/9HFrfArmbGZyDoOXGdlgG6CNf8a.jpg
25440,tt19847976,Wicked Part 2,Wicked Part 2,en,2025-12-25,"['Drama', 'Fantasy', 'Music']",['en'],0,[],0.0,0,1.995,"Wicked tells the story of Elphaba, the future ...",Planned,/iDtXSiHfHiO98oWf39AN1aVQ1d0.jpg
22146,tt21361444,Avengers: Secret Wars,Avengers: Secret Wars,en,2026-04-29,"['Action', 'Science Fiction']",['en'],0,['US'],0.0,0,37.852,An upcoming film in the Marvel Cinematic Unive...,Planned,/8chwENebfUEJzZ7sMUA0nOgiCKk.jpg
246554,tt3095356,Avatar 4,Avatar 4,en,2026-12-16,"['Action', 'Science Fiction', 'Adventure', 'Fa...",['en'],0,['US'],0.0,0,8.394,NaN,In Production,/qls9tbBTTDDEXuqm9kLB16Y7xhE.jpg


In [77]:
df_tmdb_clean[df_tmdb_clean['vote_average'] == 0]

,imdb_id,title,original_title,original_language,release_date,genres,spoken_languages,runtime,production_countries,vote_average,vote_count,popularity,overview,status,poster_path
83118,tt0276544,Survival Under Atomic Attack,Survival Under Atomic Attack,en,1951-01-01,['Documentary'],['en'],9,['US'],0.0,0,0.600,A Civil Defense film that focuses on what the ...,Released,/nTkOhT4EYW59BRdLFGXFROZRDlp.jpg
27670,tt0378616,Pyar Ki Baten,Pyar Ki Baten,hi,1951-01-01,[],[],0,[],0.0,0,0.600,Details Country of origin India Language H...,Released,/f2SkTsiESrIhwGR9KS0J8wyjJXQ.jpg
116354,tt0193737,Here Comes the Baron,Aí Vem o Barão,pt,1951-01-01,"['Music', 'Comedy']",[],83,['BR'],0.0,0,0.600,NaN,Released,/nW9EnSy5xmwYAzPuJODBE2Vx5dE.jpg
206806,tt3246414,Forest Travellers,Лесные путешественники,ru,1951-01-01,"['Animation', 'Family']",['ru'],20,['SU'],0.0,0,0.600,Every family has its scout.,Released,/aTt7MufbrWu9AdV3Z9KlAyhm2DP.jpg
27669,tt0043367,Buzdil,Buzdil,hi,1951-01-01,[],[],0,[],0.0,0,0.600,"Two brothers fall in love with the same girl, ...",Released,/bvSsnkzwdag9ooALDkprgV6daDg.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110786,tt1830809,Spaceless,Spaceless,en,2025-12-15,['Science Fiction'],['en'],0,['US'],0.0,0,2.463,A man wakes up inside a spacesuit tumbling hel...,Planned,/9HFrfArmbGZyDoOXGdlgG6CNf8a.jpg
25440,tt19847976,Wicked Part 2,Wicked Part 2,en,2025-12-25,"['Drama', 'Fantasy', 'Music']",['en'],0,[],0.0,0,1.995,"Wicked tells the story of Elphaba, the future ...",Planned,/iDtXSiHfHiO98oWf39AN1aVQ1d0.jpg
22146,tt21361444,Avengers: Secret Wars,Avengers: Secret Wars,en,2026-04-29,"['Action', 'Science Fiction']",['en'],0,['US'],0.0,0,37.852,An upcoming film in the Marvel Cinematic Unive...,Planned,/8chwENebfUEJzZ7sMUA0nOgiCKk.jpg
246554,tt3095356,Avatar 4,Avatar 4,en,2026-12-16,"['Action', 'Science Fiction', 'Adventure', 'Fa...",['en'],0,['US'],0.0,0,8.394,NaN,In Production,/qls9tbBTTDDEXuqm9kLB16Y7xhE.jpg


In [ ]:
df_test[df_test['popularity'] == 0]

,imdb_id,title,original_title,original_language,release_date,genres,spoken_languages,runtime,production_countries,vote_average,vote_count,popularity,overview,status,poster_path


In [83]:
df_test = df_tmdb_clean[(df_tmdb_clean['vote_count'] > 0) & ((df_tmdb_clean['original_language'] == 'fr') | (df_tmdb_clean['original_language'] == 'en'))]

In [86]:
print(df_test['vote_average'].median())
print(df_test['vote_average'].mean())

6.0
5.833671870768041


In [ ]:
df_tmdb_clean['score'] = df_tmdb_clean['vote_average'] * df_tmdb_clean['popularity']

0.408827862751948

In [ ]:
sns.scatterplot(data = df_test, x = 'imdb_id', y = 'popularity')

<Axes: xlabel='imdb_id', ylabel='popularity'>

Error in callback <function _draw_all_if_interactive at 0x000002500CD10A40> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

: 